In [ ]:
from pathlib import Path
import numpy as np
from airo_planner.utils import files
from pydrake.geometry import Meshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import finish_build
from airo_planner.visualization.drake_viz import add_meshcat_triad

In [ ]:
tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
scene_graph = robot_diagram_builder.scene_graph()
plant = robot_diagram_builder.plant()
builder = robot_diagram_builder.builder()
parser = robot_diagram_builder.parser()

# Add visualizer
meshcat = Meshcat()
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Load URDF files
resources_root = str(files.get_resources_dir())
ur5e_urdf = Path(resources_root) / "robots" / "ur5e" / "ur5e.urdf"
robotiq_2f_85_gripper_urdf = Path(resources_root) / "grippers" / "2f_85_gripper" / "urdf" / "robotiq_2f_85_static.urdf"
table_urdf = "table.urdf"

arm_index = parser.AddModelFromFile(str(ur5e_urdf), model_name="arm_left")
gripper_index = parser.AddModelFromFile(str(robotiq_2f_85_gripper_urdf), model_name="gripper")
table_index = parser.AddModelFromFile(str(table_urdf))

# Weld some frames together
world_frame = plant.world_frame()
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_wrist_frame = plant.GetFrameByName("wrist_3_link", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)
table_frame = plant.GetFrameByName("base_link", table_index)

plant.WeldFrames(world_frame, arm_frame)
plant.WeldFrames(arm_wrist_frame, gripper_frame)
plant.WeldFrames(world_frame, table_frame, RigidTransform([0, 0, 0]))

diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index],
    edge_step_size=0.125, # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding = 0.005,
    self_collision_padding = 0.005,
)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])

plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

In [ ]:
transform = RigidTransform(p=[0.25, 0, 0.4], rpy=RollPitchYaw([np.pi, 0, 0]))
tcp_pose_0 = np.ascontiguousarray(transform.GetAsMatrix4())

add_meshcat_triad(meshcat, "TCP Frame", X_W_Triad=transform)

In [ ]:
from typing import List
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from ur_analytic_ik import ur5e

def inverse_kinematics_fn(tcp_pose: HomogeneousMatrixType) -> List[JointConfigurationType]:
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(tcp_pose, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions

In [ ]:
solutions = inverse_kinematics_fn(tcp_pose_0)

for solution in solutions:
    print(solution, collision_checker.CheckConfigCollisionFree(solution))

In [ ]:
from cloth_tools.drake.visualization import publish_ik_solutions

publish_ik_solutions(solutions, 2.0, meshcat, diagram, context, arm_index)

In [ ]:
plant.SetPositions(plant_context, arm_index, solutions[3])
diagram.ForcedPublish(context)

In [ ]:
from cloth_tools.ompl.single_arm_planner import SingleArmOmplPlanner


planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree)
path = planner.plan_to_joint_configuration(start_joints, solutions[0])
print("Length of solution path:", planner._path_length)

In [ ]:
from cloth_tools.drake.visualization import publish_joint_path


publish_joint_path(path, 5.0, meshcat, diagram, context, arm_index)

In [ ]:
planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree)

path_lengths = []
paths = []

for solution in solutions:
    if not planner.is_state_valid_fn(solution):
        print("Solution is invalid!")
        continue
    path = planner.plan_to_joint_configuration(start_joints, solution)
    print("Length of solution path:", planner._path_length)
    paths.append(path)
    path_lengths.append(planner._path_length)

In [ ]:
for path_length in path_lengths:
    print(path_length)

shortest_path_index = np.argmin(path_lengths)
shortest_path = paths[shortest_path_index]

In [ ]:
publish_joint_path(shortest_path, 5.0, meshcat, diagram, context, arm_index)